In [ ]:
# Updated 05.10.2020

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

pandas = pd.read_table("netherlandsRP.dat")
database = db.Database("netherlands",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_CAR = Beta('ASC_CAR',0,-10,10,0)
ASC_RAIL = Beta('ASC_RAIL',0,-10,10,1)
BETA_TIME_RAIL = Beta('BETA_TIME_RAIL',0,-10,10,0)
BETA_TIME_CAR = Beta('BETA_TIME_CAR',0,-10,10,0)
BETA_COST_AGE1 = Beta('BETA_COST_AGE1',0,-10,10,0)
BETA_COST_AGE2 = Beta('BETA_COST_AGE2',0,-10,10,0)
BETA_FEMALE = Beta('BETA_FEMALE',0,-10,10,0)
BETA_FIXED_ARRIVAL_TIME = Beta('BETA_FIXED_ARRIVAL_TIME',0,-10,10,0)

# Define here arithmetic expressions for name that are not directly available from the dataxpressions for name that are not directly available from the data
rail_time  = DefineVariable('rail_time',rail_ivtt + rail_acc_time + rail_egr_time, database)
car_time = DefineVariable('car_time',car_ivtt + car_walk_time, database)

rate_G2E  = DefineVariable('rate_G2E',0.44378022,database) #from Guilders to euros
car_cost_euro  = DefineVariable('car_cost_euro',car_cost * rate_G2E,database)
rail_cost_euro  = DefineVariable('rail_cost_euro', rail_cost * rate_G2E,database)

age1 = DefineVariable('age1',(age == 0), database) #40 years or younger
age2 = DefineVariable('age2',(age == 1), database) #41 years or older
employed = DefineVariable('employed',(employ_status == 0), database)

# Utilities
V_CAR = ASC_CAR + BETA_TIME_CAR * car_time + BETA_COST_AGE1 * car_cost_euro * age1 + BETA_COST_AGE2 * car_cost_euro  * age2
V_RAIL = BETA_TIME_RAIL * rail_time + BETA_COST_AGE1 * rail_cost_euro * age1 + BETA_COST_AGE2 * rail_cost_euro * age2 + BETA_FEMALE * gender + BETA_FIXED_ARRIVAL_TIME * arrival_time

V = {0: V_CAR, 1: V_RAIL}
av = {0: 1,1: 1}


logprob = loglogit(V,av,choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "Netherlands_Base_Model"
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
display(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")